# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [52]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [53]:
import os
import shutil

In [ ]:
#/content/drive/MyDrive/Datasets/Car_Images_train/model/
os.mkdir("/content/drive/MyDrive/datasets/")
mainpath = "/content/drive/MyDrive/datasets/"

os.listdir(mainpath)
os.mkdir(mainpath + "images/")
os.mkdir(mainpath + "labels/")

In [ ]:
originpath = "/content/drive/MyDrive/money_dataset/"

In [ ]:
# !wget -O drive/MyDrive/money_dataset/money/images.zip https://drive.google.com/file/d/1rCLY3PaC-Q7z8-HzwTCXNz5_C7Dbofl6/

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
# import zipfile

In [ ]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
# money_data = zipfile.ZipFile( ## 개인별 데이터셋 압축 경로를 입력하세요 ## )

In [ ]:
# 데이터셋 압축 해제
# money_data.extractall('drive/MyDrive/money_dataset/money/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
# # 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

# !mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val
!mkdir /content/Dataset/images/test

mkdir: cannot create directory ‘/content/Dataset/images’: No such file or directory
mkdir: cannot create directory ‘/content/Dataset/images/train’: No such file or directory
mkdir: cannot create directory ‘/content/Dataset/images/val’: No such file or directory
mkdir: cannot create directory ‘/content/Dataset/labels/train’: No such file or directory
mkdir: cannot create directory ‘/content/Dataset/labels/val’: No such file or directory
mkdir: cannot create directory ‘/content/Dataset/images/test’: No such file or directory


In [ ]:
# import glob, shutil

In [ ]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = mainpath + "money/images"

In [ ]:
mainpath

'/content/drive/MyDrive/datasets/'

In [ ]:
# 이미지 폴더에 각 원 폴더 생성
for i in won_list:
    os.mkdir(mainpath + "images/" + i)

In [ ]:
# 라벨 폴더에 각 원 폴더 생성
for i in won_list:
    os.mkdir(mainpath + "labels/" + i)

In [ ]:
# origin_path에서 이미지만 추출해 mainpath로 이동
for i in won_list:
    img_list = os.listdir(originpath + i)
    for j in img_list:
        if j.endswith(".jpg") == True:
            shutil.copy((originpath + i  + '/' + j), (mainpath + 'images/' + i + '/'  + j))
    # os.move((originpath + ), )

In [ ]:
# origin_path에서 json 파일만 추출해 mainpath로 이동
for i in won_list:
    json_list = os.listdir(originpath + i)
    for j in json_list:
        if j.endswith(".json") == True:
            shutil.copy((originpath + i  + '/' + j), (mainpath + 'labels/' + i + '/'  + j))

In [ ]:
for i in won_list:
    print(i, len(os.listdir(mainpath+"labels/"+i+"/")))

10 436
50 440
100 440
500 440
1000 858
5000 867
10000 867
50000 870


---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [ ]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
mainpath

'/content/drive/MyDrive/datasets/'

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
# 3. 데이터를 Training set | Validation set으로 분할하세요.
import splitfolders
imagefrom = mainpath + "images"
destination = mainpath + "training/images"

splitfolders.ratio(imagefrom, output=destination, seed=2023, ratio=(.8, .2))

Copying files: 5178 files [01:22, 63.07 files/s]


In [ ]:
# split folders 결과 확인
print(os.listdir(destination))
print(os.listdir(destination+'/train/'))
print(os.listdir(destination+'/val/'))
print(os.listdir(destination+'/train/10/'))

['train', 'val']
['10', '50', '100', '500', '1000', '5000', '10000', '50000']
['10', '50', '100', '500', '1000', '5000', '10000', '50000']
['10_710_9.jpg', '10_1252_9.jpg', '10_416_9.jpg', '10_471_9.jpg', '10_1305_9.jpg', '10_771_1.jpg', '10_458_9.jpg', '10_666_1.jpg', '10_721_1.jpg', '10_718_9.jpg', '10_7163_9.jpg', '10_1306_9.jpg', '10_656_1.jpg', '10_23_9.jpg', '10_471_1.jpg', '10_565_9.jpg', '10_423_9.jpg', '10_1292_1.jpg', '10_790_1.jpg', '10_773_1.jpg', '10_31219_1.jpg', '10_784_1.jpg', '10_607_9.jpg', '10_1284_9.jpg', '10_800_1.jpg', '10_1303_1.jpg', '10_401_9.jpg', '10_6787_9.jpg', '10_31219_9.jpg', '10_738_9.jpg', '10_625_1.jpg', '10_780_1.jpg', '10_683_1.jpg', '10_418_9.jpg', '10_8799_1.jpg', '10_1215_9.jpg', '10_1268_9.jpg', '10_761_9.jpg', '10_673_1.jpg', '10_648_1.jpg', '10_658_1.jpg', '10_578_9.jpg', '10_1355_1.jpg', '10_690_9.jpg', '10_6637_9.jpg', '10_583_1.jpg', '10_7106_9.jpg', '10_1395_1.jpg', '10_560_9.jpg', '10_674_1.jpg', '10_6561_9.jpg', '10_7143_9.jpg', '10_794_

In [ ]:
# split folders 결과 확인
for won in won_list:
    imglist = os.listdir(destination + '/train/' + i)
    print(won, len(imglist))

10 696
50 696
100 696
500 696
1000 696
5000 696
10000 696
50000 696


In [ ]:
# 라벨 폴더에 각 원 폴더 생성
os.mkdir("/content/drive/MyDrive/datasets/training/labels/")
os.mkdir("/content/drive/MyDrive/datasets/training/labels/train/")
os.mkdir("/content/drive/MyDrive/datasets/training/labels/val/")
for i in won_list:
    os.mkdir("/content/drive/MyDrive/datasets/training/labels/train/" + i)
    os.mkdir("/content/drive/MyDrive/datasets/training/labels/val/" + i)

In [ ]:
jsonpath = mainpath+"labels/"

In [ ]:
destination, jsonpath

('/content/drive/MyDrive/datasets/training/images',
 '/content/drive/MyDrive/datasets/labels/')

In [ ]:
for won in won_list:
    print(jsonpath + won + "/")

/content/drive/MyDrive/datasets/labels/10/
/content/drive/MyDrive/datasets/labels/50/
/content/drive/MyDrive/datasets/labels/100/
/content/drive/MyDrive/datasets/labels/500/
/content/drive/MyDrive/datasets/labels/1000/
/content/drive/MyDrive/datasets/labels/5000/
/content/drive/MyDrive/datasets/labels/10000/
/content/drive/MyDrive/datasets/labels/50000/


In [ ]:
# json 파일도 똑같이 옮겨주기 (train)

for won in won_list:
    jsonlist = os.listdir(jsonpath + won)
    for json in jsonlist:
        if (json[:-4] + 'jpg') in os.listdir(destination + "/train/" + won + "/"):
            # shutil.copy("원본 json 파일", "옮길 json 파일")
            shutil.copy( (jsonpath + won + "/"  + json), ("/content/drive/MyDrive/datasets/training/labels/train/" + won + "/" + json) )

In [ ]:
# # 라벨 폴더에 각 원 폴더 생성 (val)
# for i in won_list:
#     os.mkdir("/content/drive/MyDrive/datasets/training/labels/val/" + i)

In [ ]:
# json 파일도 똑같이 옮겨주기 (val)
jsonpath = mainpath+"labels/"

for won in won_list:
    jsonlist = os.listdir(jsonpath + won)
    for json in jsonlist:
        if (json[:-4] + 'jpg') in os.listdir(destination + "/val/" + won + "/"):
            # shutil.copy("원본 json 파일", "옮길 json 파일")
            shutil.copy( (jsonpath + won + "/"  + json), ("/content/drive/MyDrive/datasets/training/labels/val/"+ won + "/" + json) )

In [ ]:
won_list

['10', '50', '100', '500', '1000', '5000', '10000', '50000']

In [ ]:
# 각 데이터 일치여부 확인
imgtrain = "/content/drive/MyDrive/datasets/training/images/train/"
imgval = "/content/drive/MyDrive/datasets/training/images/val/"
jsontrain = "/content/drive/MyDrive/datasets/training/labels/train/"
jsonval = "/content/drive/MyDrive/datasets/training/labels/val/"
def check(temp):
    for won in won_list:
        templist = os.listdir(temp + won + "/")
        print(won, ': ', len(templist))

print("train data check: ")
print(check(imgtrain), check(jsontrain))
print("val data check: ")
print(check(imgval), check(jsonval))

train data check: 
10 :  316
50 :  352
100 :  352
500 :  352
1000 :  686
5000 :  693
10000 :  693
50000 :  696
10 :  316
50 :  352
100 :  352
500 :  352
1000 :  686
5000 :  693
10000 :  693
50000 :  696
None None
val data check: 
10 :  80
50 :  88
100 :  88
500 :  88
1000 :  172
5000 :  174
10000 :  174
50000 :  174
10 :  80
50 :  88
100 :  88
500 :  88
1000 :  172
5000 :  174
10000 :  174
50000 :  174
None None


### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/4로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/4로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [ ]:
import os, json

### - 테스트하기

In [ ]:
test = "/content/drive/MyDrive/datasets/training/labels/val/1000/1000_B_DESK_0_4.json"
f = open(test)

data = json.load(f)
data

{'version': '4.5.7',
 'flags': {},
 'shapes': [{'line_color': None,
   'fill_color': None,
   'label': 'Thousand_back',
   'points': [[502.71428571428555, 879.9999999999999],
    [2505.5714285714284, 1879.9999999999998]],
   'group_id': None,
   'shape_type': 'rectangle',
   'flags': {}}],
 'imagePath': '1000_B_DESK_0_4.jpg',
 'imageData': None,
 'imageHeight': 3024,
 'imageWidth': 3024,
 'lineColor': [0, 255, 0, 128],
 'fillColor': [255, 0, 0, 128]}

In [ ]:
data['shapes'][0]['points'], [i/4 for i in data['shapes'][0]['points'][0]],  [i/4 for i in data['shapes'][0]['points'][1]]

([[502.71428571428555, 879.9999999999999],
  [2505.5714285714284, 1879.9999999999998]],
 [125.67857142857139, 219.99999999999997],
 [626.3928571428571, 469.99999999999994])

In [ ]:
(data['shapes'][0]["label"]).split('_'), (data['shapes'][0]["label"]).split('_')[0]

(['Thousand', 'back'], 'Thousand')

In [ ]:
xlabels = [i/5 for i in data['shapes'][0]['points'][0]]
ylabels = [i/5 for i in data['shapes'][0]['points'][1]]

x1 = [i/5 for i in data['shapes'][0]['points'][0]][0]
y1 = [i/5 for i in data['shapes'][0]['points'][0]][1]
x2 = [i/5 for i in data['shapes'][0]['points'][1]][0]
y2 = [i/5 for i in data['shapes'][0]['points'][1]][1]

x1, x2, y1, y2

(100.54285714285712, 501.1142857142857, 175.99999999999997, 375.99999999999994)

In [ ]:
# Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]

def yolotxter(jsonfile):
    f = open(jsonfile)
    jsonread = json.load(f)
    x1 = [i/5 for i in jsonread['shapes'][0]['points'][0]][0]
    y1 = [i/5 for i in jsonread['shapes'][0]['points'][0]][1]
    x2 = [i/5 for i in jsonread['shapes'][0]['points'][1]][0]
    y2 = [i/5 for i in jsonread['shapes'][0]['points'][1]][1]

    templabel = (jsonread['shapes'][0]["label"]).split('_')
    label_list = [i for i in templabel[:-1]]
    label = ""
    labeldic = {    "Ten": 0, "Fifty": 1, "Hundred": 2, "Five_Hundred": 3, "Thousand": 4, "Five_Thousand": 5, "Ten_Thousand": 6, "Fifty_Thousand": 7}
    for j in label_list:
        label += j
        label += '_'
    label = label[:-1]

    yolotxt = f'{labeldic[label]} {(x1 + x2)/2} {(y1 + y2)/2} {x2 - x1} {y2 - y1}'

    return yolotxt
# yolotxter(test)

### - 직접 해보기

In [ ]:
# 가장 먼저! txt 파일이 들어갈 폴더 생성해주기
os.mkdir('/content/drive/MyDrive/datasets/training/labeltxt/')
os.mkdir('/content/drive/MyDrive/datasets/training/labeltxt/train/')
os.mkdir('/content/drive/MyDrive/datasets/training/labeltxt/val/')

for won in won_list:
    os.mkdir(f'/content/drive/MyDrive/datasets/training/labeltxt/train/{won}/')
    os.mkdir(f'/content/drive/MyDrive/datasets/training/labeltxt/val/{won}/')

In [ ]:
p = '/content/drive/MyDrive/datasets/training/labeltxt/train/10/10_1215_1.json'
p[:-4]

'/content/drive/MyDrive/datasets/training/labeltxt/train/10/10_1215_1.'

In [ ]:
# train에 txt 파일 생성해주기

for won in won_list: 
    train_jsons = os.listdir('/content/drive/MyDrive/datasets/training/labels/train/' + won)

    for file in train_jsons:
        temp_path = '/content/drive/MyDrive/datasets/training/labels/train/' + f'{won}/' + file
        txtout = yolotxter(temp_path)
        f = open(f"/content/drive/MyDrive/datasets/training/labeltxt/train/{won}/{file[:-4]}txt", "w")
        f.write(txtout)
        f.close()

In [ ]:
# val에 txt 파일 생성해주기

for won in won_list: 
    val_jsons = os.listdir('/content/drive/MyDrive/datasets/training/labels/val/' + won)

    for file in val_jsons:
        temp_path = '/content/drive/MyDrive/datasets/training/labels/val/' + f'{won}/' + file
        txtout = yolotxter(temp_path)
        f = open(f"/content/drive/MyDrive/datasets/training/labeltxt/val/{won}/{file[:-4]}txt", "w")
        f.write(txtout)
        f.close()

In [ ]:
# # YOLO가 인식할 수 있는 형태로 폴더 바꿔주기
# os.rename("/content/drive/MyDrive/datasets/training/labels/", "/content/drive/MyDrive/datasets/training/json_labels/")
# os.rename("/content/drive/MyDrive/datasets/training/labeltxt/", "/content/drive/MyDrive/datasets/training/labels/")

In [ ]:
# # 50000 안옮겨진 부분 고쳐주기
# #json

# jsonlist = os.listdir(jsonpath + "50000")
# for json in jsonlist:
#     if (json[:-4] + 'jpg') in os.listdir(destination + "/train/50000/"):
#         # shutil.copy("원본 json 파일", "옮길 json 파일")
#         shutil.copy( (jsonpath + "50000/"  + json), ("/content/drive/MyDrive/datasets/training/json_labels/train/50000/" + json) )

In [ ]:
# # os.remove('/content/drive/MyDrive/datasets/training/labels/train/5000/')

# for i in os.listdir('/content/drive/MyDrive/datasets/training/labels/train/5000/'):
#     os.remove('/content/drive/MyDrive/datasets/training/labels/train/5000/' + i)

In [ ]:
# #txt
# train_jsons = os.listdir('/content/drive/MyDrive/datasets/training/labels/train/5000/')

# for file in train_jsons:
#     temp_path = '/content/drive/MyDrive/datasets/labels/5000/' + file
#     txtout = yolotxter(temp_path)
#     f = open(f"/content/drive/MyDrive/datasets/training/labels/train/5000/{file[:-4]}txt", "w")
#     f.write(txtout)
#     f.close()

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
# # yaml example

# path: /content/datasets/street  
# train: images/train  
# val: images/train  
# test:  

# nc: 3  
# names: ['person', 'vehicle', 'person_in_vehicle'] 

In [ ]:
import yaml

In [ ]:
won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}

In [ ]:
won_list

['10', '50', '100', '500', '1000', '5000', '10000', '50000']

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################

with open('/content/drive/MyDrive/datasets/training/money.yaml', 'w') as f :
    writing = '''
    path: /content/drive/MyDrive/datasets/training/
    train: images/train
    val: images/val
    test:
    nc: 8
    names: ["Ten", "Fifty", "Hundred", "Five_Hundred", "Thousand", "Ten_Thousand", "Fifty_Thousand"]
    '''
    f.write(writing)
    f.close()

In [55]:
# 각 데이터 일치여부 확인
imgtrain = "/content/drive/MyDrive/datasets/training/images/train/"
imgval = "/content/drive/MyDrive/datasets/training/images/val/"
jsontrain = "/content/drive/MyDrive/datasets/training/labels/train/"
jsonval = "/content/drive/MyDrive/datasets/training/labels/val/"
def check(temp):
    answer = 0
    for won in won_list:
        templist = os.listdir(temp + won + "/")
        answer += len(templist)
    return answer
print("train data check: ")
print(check(imgtrain))
print("train txt check: ")
print(check(jsontrain))
print("val data check: ")
print(check(imgval))
print("val txt check: ")
print(check(jsonval))

train data check: 
4013
train txt check: 
4140
val data check: 
1038
val txt check: 
1038
